Monte Carlo 학습 구현

In [1]:
import random

In [4]:
#Grid world class
class GridWorld():
    def __init__(self):
        self.x=0
        self.y=0
    
    #Action
    def step(self, a): #Agent로부터 Action을 받아서 상태 변이를 일으키고 보상을 정해주는 함수
        if a == 0:
            self.move_right()
        elif a == 1:
            self.move_left()
        elif a == 2:
            self.move_up()
        elif a == 3:
            self.move_down()
    
        reward = -1 #어느 방향으로 가든 무조건 보상 -1
        done = self.is_done() #에피소드가 끝났는 지 알려주는 함수
        return (self.x, self.y), reward, done

    def move_right(self):
        self.y += 1
        if self.y > 3:
            self.y = 3
        
    def move_left(self):
        self.y -= 1
        if self.y < 0:
            self.y = 0
    
    def move_up(self):
        self.x -= 1
        if self.x < 0:
            self.x = 0

    def move_down(self):
        self.x += 1
        if self.x > 3:
            self.x = 3

    def is_done(self):
        if self.x == 3 and self.y == 3: #(3,3)-도착지점에 도착하면 True 아닐 시 False
            return True
        else:
            return False

    def get_state(self):
        return(self.x, self.y)

    def reset(self): #도착지점에 도달 시 처음 상태로 돌아가기 위해 위치 reset
        self.x = 0
        self.y = 0
        return(self.x, self.y)

In [13]:
#Agent Class
class Agent():
    def __init__(self):
        pass
    
    def select_action(self): #Agent가 네 방향으로 향하는 확률이 모두 동일하기 때문에 설정이 쉬움(아래 코드 참조)
        coin = random.random()
        if coin < 0.25:
            action = 0
        elif coin < 0.5:
            action = 1
        elif coin < 0.75:
            action = 2
        else:
            action = 3
        return action


In [18]:
#Main 함수
def main():
    env = GridWorld() #기존에 구축한 GridWorld를 환경으로 선언
    agent = Agent() #Agent 선언
    data = [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]] #테이블 초기화
    gamma = 1.0
    alpha = 0.0001

    for k in range(50000): # 에피소드 진행 수 (Agent가 경험을 쌓는 과정)
        done = False
        history = []
        while not done:
            action = agent.select_action()
            (x,y), reward, done = env.step(action)
            history.append((x,y,reward))
        env.reset()

        #매 에피소드가 끝나고 바로 해당 데이터를 이용해 테이블을 업데이트
        cum_reward = 0 #Return(Gt)
        for transition in history[::-1]:
            #방문했던 상태들을 뒤에서부터 보며 차례차례 리턴을 계산
            x, y, reward = transition
            data[x][y] = data[x][y] + alpha*(cum_reward-data[x][y])
            cum_reward = cum_reward + gamma*reward

    #학습이 끝나고 난 후 데이터를 출력해보기 위한 코드
    for row in data:
        print(row)

In [17]:
main() #10000번 진행 시

[-58.960320376619414, -57.637937192441555, -54.179227295487614, -50.43379631413986]
[-56.79424504786019, -53.6771386655389, -48.39964974904007, -42.89063837708747]
[-52.87814584937751, -47.76176454223333, -37.53512708426639, -25.625137397394273]
[-49.386947333782636, -42.51683038907416, -25.65218788502933, 0.0]


In [19]:
main() #에피소드 50000번 진행

[-58.87996734442347, -56.91230004592524, -54.182616839454646, -51.6908488752423]
[-56.9307120769497, -53.41797859110891, -48.85198934430854, -44.904140800672266]
[-53.317856297612515, -48.38795633428589, -39.82394910009695, -30.092740783888104]
[-50.95835364909059, -43.78369885020527, -28.86545252039, 0.0]


에피소드 진행이 길어질 수록 실제 값에 가까워지는 것을 확인 가능

# Temporal Difference
### Monte carlo의 학습 방식은 치명적인 단점이 존재한다.
#### 하나의 Episode가 끝나야지만 업데이트가 이루어진다는 것.
업데이트를 하기 위해서 Return 값을 필요로 하기 때문인데 즉, 
MC는 적용할 수 있는 환경이 매우 제한적이라는 것이다.
하지만 현실에서는 뚜렷한 목적이 존재하지 않는 경우가 더 많이 때문에 MC는 적합하다고 볼 수 없는데 이에 대한 해결책이 바로 TD 방법론이다.

TD는 간단하게 설명하면 미래의 추측을 통해 과거의 추측을 변경하는 방법인데 이해하기 어려우니 예시를 들어보자.
금요일(일요일 강수확률 50%) / 토요일(일요일 강수확률 80%)라고 가정을 해보자
이렇게 일요일 강수확률이 금요일과 토요일 각각 존재한다고 할 때 당연하게도 토요일이라는 정보가 추가된 토요일의 예측이 더 정확할 것이다.
이러한 경우처럼 토요일의 추측을 통해 금요일의 추측을 변경하는 것으로 이러한 방법론을 적용하는 경우에는 목표지점을 알지 못 하더라도
값을 업데이트 하는 것이 가능하다.

In [20]:
#Temporal Difference 학습 구현
def TD_main():
    env = GridWorld()
    agent = Agent()
    data = [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]]
    gamma = 1.0
    alpha = 0.01 #MC보다 크게 적용

    for k in range(50000):
        done = False
        while not done:
            x, y = env.get_state()
            action = agent.select_action()
            (x_prime, y_prime), reward, done = env.step(action)

            #한번의 Step이 진행되자 마자 바로 테이블의 데이터를 업데이트 해줌
            data[x][y] = data[x][y] + alpha*(reward+gamma*data[x_prime][y_prime]-data[x][y])
        env.reset()

    for row in data:
        print(row)

In [21]:
TD_main()

[-59.289709615001996, -57.59948891273813, -54.01352760770926, -50.38131071503045]
[-57.11021750732072, -54.00920503216625, -48.98453392928103, -43.558415925425244]
[-54.22446772991143, -48.71689927795067, -38.246364890967584, -26.411818129081418]
[-52.21700927997843, -45.14756456121253, -28.894331937603805, 0]


# MC VS TD
## 1. Episodic MDP 
### 종료 상태가 존재해 Agent의 경험이 Episode 단위로 나뉘어지는 것
## 2. Non-Episodic MDP
### 종료 상태 없이 하나의 EPisode가 무한히 이어지는 MDP
# 
#### MC의 경우에는 1번의 상황에만 적용이 가능하지만 그만큼 Episode 진행이 많아질수록 절대 틀릴 수 없는 방법론(안정적인 방법)
#### TD는 1번과 2번 어디에나 적용이 가능하다는 장점이 존재!(편향성이 존재하지만 Deep RL에서 어느정도 해결이 가능)
## 분산의 측면
## MC - 하나의 Episode가 목적지에 도달하기 까지 수 많은 변수가 존재하기 때문에 분산이 매우 크게 나타남
## TD - 하나의 Sample만을 참고해 업데이트를 하기 때문에 분산이 매우 작음
## *어떤 방법론이 더 좋다라고 구분할 수는 없지만 상황에 따라 각기의 존재함.
